# Notebook d'exploration des données issues des pipelines SmartTech :
- delta_bronze : fichier Delta issue du pipeline en direct depuis les fichiers json
- delta_bronze_kafka : fichier Delta issue du pipeline en sortie de Kafka
- delta_silver_kafka : fichier Delta issue de la transformation de delta_bronze_kafka


In [1]:
##########################################################
###### SCRIPT POUR CONFIGURER SPARK AVEC DELTA LAKE ######
##########################################################

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp
from pathlib import Path
from delta import configure_spark_with_delta_pip


builder = (
    SparkSession.builder.appName("SmartTech_Exploration")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)


spark = configure_spark_with_delta_pip(builder).getOrCreate()

print(f"Spark version : {spark.version}")
print("Connecteur Delta chargé avec succès.")


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/17 16:00:40 WARN Utils: Your hostname, MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 10.0.21.53 instead (on interface en0)
25/12/17 16:00:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/jean-thomasmiquelot/kDrive/PROGRAMMATION/simplon/Simplon_projets/esther_spark_streaming/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/jean-thomasmiquelot/.ivy2.5.2/cache
The jars for the packages stored in: /Users/jean-thomasmiquelot/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-820e613a-350d-4de0-8a70-240fcb1b9be2;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antl

Spark version : 4.0.1
Connecteur Delta chargé avec succès.


# Exploration Table Delta Pipeline Classique

In [2]:
# Charger les données Delta Lake depuis le répertoire bronze
PROJECT_ROOT = Path.cwd().parent
BRONZE_PATH = PROJECT_ROOT / "data" / "out" / "delta_bronze"
df_bronze = spark.read.format("delta").load(str(BRONZE_PATH))

In [9]:
# Afficher les 20 premières lignes du DataFrame bronze
df_bronze.show(20, truncate=False)

+-------------------+-----------------+--------+-----+------------------+------+----+---------------+
|timestamp          |device_id        |building|floor|type              |value |unit|_corrupt_record|
+-------------------+-----------------+--------+-----+------------------+------+----+---------------+
|2025-01-12 09:27:25|sensor-temp-003  |B       |3    |temperature       |21.6  |°C  |NULL           |
|2025-01-12 09:27:28|sensor-temp-003  |A       |3    |temperature       |18.6  |°C  |NULL           |
|2025-01-12 09:27:29|sensor-temp-003  |B       |2    |temperature       |24.1  |°C  |NULL           |
|2025-01-12 09:27:30|sensor-energy-010|B       |3    |energy_consumption|172.7 |kWh |NULL           |
|2025-01-12 09:27:31|sensor-temp-001  |A       |2    |temperature       |18.2  |°C  |NULL           |
|2025-01-12 09:28:18|sensor-hum-002   |B       |3    |humidity          |59.1  |%   |NULL           |
|2025-01-12 09:28:20|sensor-temp-001  |A       |1    |temperature       |26.5  |°C

In [ ]:
# Créer une vue temporaire pour exécuter des requêtes SQL
df_bronze.createOrReplaceTempView("bronze_table")

In [6]:
# Exemple de requête SQL pour explorer les données
spark.sql("SELECT * from bronze_table LIMIT 10").show()

+-------------------+-----------------+--------+-----+------------------+------+----+---------------+
|          timestamp|        device_id|building|floor|              type| value|unit|_corrupt_record|
+-------------------+-----------------+--------+-----+------------------+------+----+---------------+
|2025-01-12 09:27:25|  sensor-temp-003|       B|    3|       temperature|  21.6|  °C|           NULL|
|2025-01-12 09:27:28|  sensor-temp-003|       A|    3|       temperature|  18.6|  °C|           NULL|
|2025-01-12 09:27:29|  sensor-temp-003|       B|    2|       temperature|  24.1|  °C|           NULL|
|2025-01-12 09:27:30|sensor-energy-010|       B|    3|energy_consumption| 172.7| kWh|           NULL|
|2025-01-12 09:27:31|  sensor-temp-001|       A|    2|       temperature|  18.2|  °C|           NULL|
|2025-01-12 09:38:21|  sensor-temp-001|       B|    1|       temperature|  20.3|  °C|           NULL|
|2025-01-12 09:38:23|  sensor-temp-001|       B|    2|       temperature|  23.3|  

In [8]:
# Compter le nombre total d'enregistrements dans la table bronze
# Possibilité de facilement modifier cette explorations SQL
sql_query = "SELECT COUNT(*) AS total_records FROM bronze_table"
spark.sql(sql_query).show()

+-------------+
|total_records|
+-------------+
|          500|
+-------------+



# Exploration Table Delta Pipeline avec Kafka

In [2]:
PROJECT_ROOT = Path.cwd().parent
BRONZE_KAFKA_PATH = PROJECT_ROOT / "data" / "out" / "delta_bronze_kafka"
SILVER_KAFKA_PATH = PROJECT_ROOT / "data" / "out" / "delta_silver_kafka"

In [3]:
df_bronze_kafka = spark.read.format("delta").load(str(BRONZE_KAFKA_PATH))
df_silver_kafka = spark.read.format("delta").load(str(SILVER_KAFKA_PATH))

## Exploration delta_bronze_kafka

In [6]:
df_bronze_kafka.show(20, truncate=False)

25/12/17 14:56:31 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------------+-----------------+--------+-----+------------------+-----+----+---------------+
|timestamp          |device_id        |building|floor|type              |value|unit|_corrupt_record|
+-------------------+-----------------+--------+-----+------------------+-----+----+---------------+
|2025-01-12 09:39:53|sensor-hum-002   |B       |2    |humidity          |44.2 |%   |NULL           |
|2025-01-12 09:40:07|sensor-temp-001  |A       |1    |temperature       |26.9 |°C  |NULL           |
|2025-01-12 09:40:23|sensor-hum-002   |B       |2    |humidity          |37.9 |%   |NULL           |
|2025-01-12 09:40:28|sensor-temp-003  |B       |1    |temperature       |19.3 |°C  |NULL           |
|2025-01-12 09:40:47|sensor-hum-002   |B       |3    |humidity          |40.5 |%   |NULL           |
|2025-01-12 09:41:05|sensor-co2-020   |A       |1    |co2               |744.0|ppm |NULL           |
|2025-01-12 09:41:22|sensor-temp-001  |A       |1    |temperature       |24.3 |°C  |NULL   

In [4]:
# afficher en pandas (plus propre)
# possible seulement si le dataset est petit
df_bronze_kafka.limit(500).toPandas()

25/12/17 16:01:00 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,timestamp,device_id,building,floor,type,value,unit,_corrupt_record
0,2025-01-12 09:18:00,sensor-co2-020,B,1,co2,862.000000,ppm,None
1,2025-01-12 09:18:04,sensor-temp-003,B,3,temperature,27.500000,°C,None
2,2025-01-12 09:18:07,sensor-temp-001,A,2,temperature,26.500000,°C,None
3,2025-01-12 09:18:12,sensor-hum-002,B,1,humidity,30.299999,%,None
4,2025-01-12 09:18:14,sensor-co2-020,A,2,co2,1008.000000,ppm,None
...,...,...,...,...,...,...,...,...
495,2025-01-12 09:32:00,sensor-temp-003,A,3,temperature,27.900000,°C,None
496,2025-01-12 09:40:28,sensor-temp-003,B,1,temperature,19.299999,°C,None
497,2025-01-12 09:40:33,sensor-co2-020,A,3,co2,420.000000,ppm,None
498,2025-01-12 09:37:18,sensor-hum-002,B,1,humidity,58.900002,%,None


In [5]:
df_bronze_kafka.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- device_id: string (nullable = true)
 |-- building: string (nullable = true)
 |-- floor: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- value: float (nullable = true)
 |-- unit: string (nullable = true)
 |-- _corrupt_record: string (nullable = true)



In [6]:
df_bronze_kafka.describe().show()

+-------+---------------+--------+------------------+-----------+------------------+----+---------------+
|summary|      device_id|building|             floor|       type|             value|unit|_corrupt_record|
+-------+---------------+--------+------------------+-----------+------------------+----+---------------+
|  count|            500|     500|               500|        500|               500| 500|              0|
|   mean|           NULL|    NULL|             2.006|       NULL|223.44339991760253|NULL|           NULL|
| stddev|           NULL|    NULL|0.8168834661041859|       NULL|338.76289488129237|NULL|           NULL|
|    min| sensor-co2-020|       A|                 1|        co2|              18.1|   %|           NULL|
|    max|sensor-temp-003|       B|                 3|temperature|            1200.0|  °C|           NULL|
+-------+---------------+--------+------------------+-----------+------------------+----+---------------+



In [7]:
df_bronze_kafka.count()

500

In [8]:
df_bronze_kafka.createOrReplaceTempView("kafka_bronze_table")

In [ ]:
spark.sql("SELECT * from kafka_bronze_table LIMIT 10").show()

+-------------------+---------------+--------+-----+-----------+------+----+---------------+
|          timestamp|      device_id|building|floor|       type| value|unit|_corrupt_record|
+-------------------+---------------+--------+-----+-----------+------+----+---------------+
|2025-01-12 09:34:35| sensor-co2-020|       B|    3|        co2| 817.0| ppm|           NULL|
|2025-01-12 09:34:36|sensor-temp-003|       B|    3|temperature|  23.2|  °C|           NULL|
|2025-01-12 09:34:40|sensor-temp-003|       B|    1|temperature|  19.9|  °C|           NULL|
|2025-01-12 09:34:42| sensor-co2-020|       B|    2|        co2|1051.0| ppm|           NULL|
|2025-01-12 09:34:44| sensor-hum-002|       B|    2|   humidity|  39.5|   %|           NULL|
|2025-01-12 09:42:44| sensor-co2-020|       B|    3|        co2| 571.0| ppm|           NULL|
|2025-01-12 09:42:48| sensor-hum-002|       A|    3|   humidity|  42.7|   %|           NULL|
|2025-01-12 09:42:50| sensor-co2-020|       A|    3|        co2| 922.0

## Exploration delta_silver_kafka

In [9]:
df_silver_kafka.show(20, truncate=False)

+-------------------+-----------------+--------+-----+------------------+------+----+
|timestamp          |device_id        |building|floor|type              |value |unit|
+-------------------+-----------------+--------+-----+------------------+------+----+
|2025-01-12 09:31:21|sensor-energy-010|a       |2    |energy_consumption|184.0 |kwh |
|2025-01-12 09:34:57|sensor-energy-010|b       |3    |energy_consumption|163.2 |kwh |
|2025-01-12 09:21:16|sensor-energy-010|b       |3    |energy_consumption|190.6 |kwh |
|2025-01-12 09:25:22|sensor-energy-010|b       |2    |energy_consumption|134.0 |kwh |
|2025-01-12 09:32:58|sensor-temp-001  |b       |2    |temperature       |20.9  |°c  |
|2025-01-12 09:39:04|sensor-temp-003  |a       |1    |temperature       |23.7  |°c  |
|2025-01-12 09:20:52|sensor-temp-003  |b       |3    |temperature       |21.0  |°c  |
|2025-01-12 09:28:41|sensor-temp-003  |a       |3    |temperature       |18.9  |°c  |
|2025-01-12 09:24:57|sensor-temp-003  |b       |2    |

In [10]:
# afficher en pandas (plus propre)
# possible seulement si le dataset est petit
df_silver_kafka.toPandas()

,timestamp,device_id,building,floor,type,value,unit
0,2025-01-12 09:31:21,sensor-energy-010,a,2,energy_consumption,184.000000,kwh
1,2025-01-12 09:34:57,sensor-energy-010,b,3,energy_consumption,163.199997,kwh
2,2025-01-12 09:21:16,sensor-energy-010,b,3,energy_consumption,190.600006,kwh
3,2025-01-12 09:25:22,sensor-energy-010,b,2,energy_consumption,134.000000,kwh
4,2025-01-12 09:29:49,sensor-energy-010,a,3,energy_consumption,142.899994,kwh
...,...,...,...,...,...,...,...
495,2025-01-12 09:42:55,sensor-temp-003,b,2,temperature,21.900000,°c
496,2025-01-12 09:31:16,sensor-temp-003,b,3,temperature,22.400000,°c
497,2025-01-12 09:31:10,sensor-co2-020,a,2,co2,838.000000,ppm
498,2025-01-12 09:41:42,sensor-hum-002,b,3,humidity,49.599998,%


In [11]:
df_silver_kafka.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- device_id: string (nullable = true)
 |-- building: string (nullable = true)
 |-- floor: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- value: float (nullable = true)
 |-- unit: string (nullable = true)



In [12]:
df_silver_kafka.describe().show()

+-------+---------------+--------+-----------------+-----------+------------------+----+
|summary|      device_id|building|            floor|       type|             value|unit|
+-------+---------------+--------+-----------------+-----------+------------------+----+
|  count|            500|     500|              500|        500|               500| 500|
|   mean|           NULL|    NULL|            2.006|       NULL|223.44339991760253|NULL|
| stddev|           NULL|    NULL|0.816883466104186|       NULL|338.76289488129237|NULL|
|    min| sensor-co2-020|       a|                1|        co2|              18.1|   %|
|    max|sensor-temp-003|       b|                3|temperature|            1200.0|  °c|
+-------+---------------+--------+-----------------+-----------+------------------+----+



In [13]:
df_silver_kafka.createOrReplaceTempView("kafka_silver_table")
spark.sql("SELECT * from kafka_silver_table LIMIT 10").show()

+-------------------+-----------------+--------+-----+------------------+------+----+
|          timestamp|        device_id|building|floor|              type| value|unit|
+-------------------+-----------------+--------+-----+------------------+------+----+
|2025-01-12 09:31:21|sensor-energy-010|       a|    2|energy_consumption| 184.0| kwh|
|2025-01-12 09:34:57|sensor-energy-010|       b|    3|energy_consumption| 163.2| kwh|
|2025-01-12 09:32:58|  sensor-temp-001|       b|    2|       temperature|  20.9|  °c|
|2025-01-12 09:39:04|  sensor-temp-003|       a|    1|       temperature|  23.7|  °c|
|2025-01-12 09:24:57|  sensor-temp-003|       b|    2|       temperature|  19.7|  °c|
|2025-01-12 09:22:01|   sensor-hum-002|       a|    2|          humidity|  32.2|   %|
|2025-01-12 09:35:07|   sensor-co2-020|       a|    2|               co2|1024.0| ppm|
|2025-01-12 09:39:01|   sensor-co2-020|       a|    3|               co2| 645.0| ppm|
|2025-01-12 09:41:42|   sensor-hum-002|       b|    3|

In [14]:
# Compter le nombre total d'enregistrements dans la table silver kafka
# Possibilité de facilement modifier cette explorations SQL
silver_sql_query = "SELECT COUNT(*) AS total_records FROM kafka_silver_table"
spark.sql(silver_sql_query).show()

+-------------+
|total_records|
+-------------+
|          500|
+-------------+



In [15]:
spark.sql("SELECT DISTINCT (unit) as unit FROM kafka_silver_table").show()

+----+
|unit|
+----+
| kwh|
|  °c|
| ppm|
|   %|
+----+

